In [23]:
#diabetes df
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns
import ipywidgets
from ipywidgets import widgets as wd
import sklearn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import metrics as mt
from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline

In [2]:
df_mlt = pd.read_csv('/Users/mtran/Documents/SMUDataSciences/2019-Summer/MiniLab/Diabetic_Cleaned.csv')
print("Finished loading data")

Finished loading data


In [3]:
#make dataframes for each column
ordinal = ["age", "time_in_hospital"]
categorical = ["race", "gender", "admission_type_id", "discharge_disposition_id", "admission_source_id", "medical_specialty", "max_glu_serum", "A1Cresult", "change", "diabetesMed", "diag_1_val", "diag_2_val", "diag_3_val", "meds_increased"]
continuous = ["num_lab_procedures", "num_procedures", "num_medications", "number_outpatient", "number_emergency", "number_inpatient", "number_diagnoses", "medication_count"]
medicines = ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "insulin", "glyburide-metformin", "glipizide-metformin", "metformin-rosiglitazone", "metformin-pioglitazone"]
response = ["readmitted_tf"]

In [4]:
df_mlt_ordinal = df_mlt[ordinal]
df_mlt_categorical = df_mlt[categorical]
df_mlt_continuous = df_mlt[continuous]
df_mlt_medicines = df_mlt[medicines]
df_mlt_response = df_mlt[response]

In [5]:
#Make age ordinal
for i in range(0, 10):
    val = "[" + str(i*10) + "-" + str((i+1)*10) + ")"
    df_mlt_ordinal.loc[df_mlt_ordinal['age'] == val, 'age'] = i

In [6]:
#Make all categorical columns categorical
df_mlt_categorical['admission_type_id'] = df_mlt_categorical['admission_type_id'].astype(object)
df_mlt_categorical['discharge_disposition_id'] = df_mlt_categorical['discharge_disposition_id'].astype(object)
df_mlt_categorical['admission_source_id'] = df_mlt_categorical['admission_source_id'].astype(object)
df_mlt_categorical['admission_type_id'] = df_mlt_categorical['admission_type_id'].astype(object)
df_mlt_categorical['meds_increased'] = df_mlt_categorical['meds_increased'].astype(object)
print("Done")

Done


In [7]:
#Creating X and Y dataframes
df_mlt_x = pd.concat([df_mlt_ordinal, df_mlt_categorical, df_mlt_continuous], axis = 1)
df_mlt_med_x = pd.concat([df_mlt_ordinal, df_mlt_categorical, df_mlt_continuous, df_mlt_medicines], axis = 1)
df_mlt_hot_x = pd.concat([df_mlt_ordinal, pd.get_dummies(df_mlt_categorical), df_mlt_continuous], axis = 1)
df_mlt_hot_med_x = pd.concat([df_mlt_ordinal, pd.get_dummies(df_mlt_categorical), df_mlt_continuous, pd.get_dummies(df_mlt_medicines)], axis = 1)
df_mlt_y = df_mlt_response
print("Done")

Done


In [8]:
#Attempted to log all numerical columns ---ended up with only taking the log for 3

#medication_count contains 0
#df_mlt_continuous["medication_count"][df_mlt_continuous["medication_count"]==0]=0.0001 #add small value to 0
#df_mlt_continuous["medication_count"]=np.nan
#df_mlt_continuous["medication_count"]=np.log(df_mlt_continuous["medication_count"])

#num_lab_procedures
df_mlt_continuous["num_lab_procedures"][df_mlt_continuous["num_lab_procedures"]==0]=0.0001 #add small value to 0
df_mlt_continuous["num_lab_procedures"]=np.nan
df_mlt_continuous["num_lab_procedures"]=np.log(df_mlt_continuous["num_lab_procedures"])

#num_medications
df_mlt_continuous["num_medications"][df_mlt_continuous["num_medications"]==0]=0.0001 #add small value to 0
df_mlt_continuous["num_medications"]=np.nan
df_mlt_continuous["num_medications"]=np.log(df_mlt_continuous["num_medications"])

#num_procedures contains 0
#df_mlt_continuous["num_procedures"][df_mlt_continuous["num_procedures"]==0]=0.0001 #add small value to 0
#df_mlt_continuous["num_procedures"]=np.nan
#df_mlt_continuous["num_procedures"]=np.log(df_mlt_continuous["num_procedures"])

#num_diagnoses
df_mlt_continuous["number_diagnoses"][df_mlt_continuous["number_diagnoses"]==0]=0.0001 #add small value to 0
df_mlt_continuous["number_diagnoses"]=np.nan
df_mlt_continuous["number_diagnoses"]=np.log(df_mlt_continuous["number_diagnoses"])

#number_emergency contains 0
#df_mlt_continuous["number_emergency"][df_mlt_continuous["number_emergency"]==0]=0.0001 #add small value to 0
#df_mlt_continuous["number_emergency"]=np.nan
#df_mlt_continuous["number_emergency"]=np.log(df_mlt_continuous["number_emergency"])

#number_inpatient contains 0
#df_mlt_continuous["number_inpatient"][df_mlt_continuous["number_inpatient"]==0]=0.0001 #add small value to 0
#df_mlt_continuous["number_inpatient"]=np.nan
#df_mlt_continuous["number_inpatient"]=np.log(df_mlt_continuous["number_inpatient"])

#number_outpatient contains 0
#df_mlt_continuous["number_outpatient"][df_mlt_continuous["number_outpatient"]==0]=0.0001 #add small value to 0
#df_mlt_continuous["number_outpatient"]=np.nan
#df_mlt_continuous["number_outpatient"]=np.log(df_mlt_continuous["number_outpatient"])


#Inital attempt to log all numerical values
#df_mlt_continuous['lmedication_count'] = np.log(df_mlt_continuous["medication_count"])
#df_mlt_continuous['lnum_lab_procedures'] = np.log(df_mlt_continuous["num_lab_procedures"])
#df_mlt_continuous['lnum_medications'] = np.log(df_mlt_continuous["num_medications"])
#df_mlt_continuous['lnum_procedures'] = np.log(df_mlt_continuous["num_procedures"])
#df_mlt_continuous['lnumber_diagnoses'] = np.log(df_mlt_continuous["number_diagnoses"])
#df_mlt_continuous['lnumber_emergency'] = np.log(df_mlt_continuous["number_emergency"])
#df_mlt_continuous['lnumber_inpatient'] = np.log(df_mlt_continuous["number_inpatient"])
#df_mlt_continuous['lnumber_outpatient'] = np.log(df_mlt_continuous["number_outpatient"])

#df_mlt_lcontinuous = pd.concat((df_mlt_continuous['lmedication_count'],df_mlt_continuous['lnum_lab_procedures'],
                            #df_mlt_continuous['lnum_medications'],df_mlt_continuous['lnum_procedures'],
                            #df_mlt_continuous['lnumber_diagnoses'],df_mlt_continuous['lnumber_emergency'],
                            #df_mlt_continuous['lnumber_inpatient'],df_mlt_continuous['lnumber_outpatient']),axis=1)
print("Done")

Done


In [11]:
df_mlt_hot = pd.concat([df_mlt_hot_x,df_mlt_y], axis=1)
df_mlt_hot.dtypes

age                            int64
time_in_hospital               int64
race_AfricanAmerican           uint8
race_Asian                     uint8
race_Caucasian                 uint8
race_Hispanic                  uint8
race_Other                     uint8
race_Unspecified               uint8
gender_Female                  uint8
gender_Male                    uint8
admission_type_id_1            uint8
admission_type_id_2            uint8
admission_type_id_3            uint8
admission_type_id_4            uint8
admission_type_id_5            uint8
admission_type_id_6            uint8
admission_type_id_7            uint8
admission_type_id_8            uint8
discharge_disposition_id_1     uint8
discharge_disposition_id_2     uint8
discharge_disposition_id_3     uint8
discharge_disposition_id_4     uint8
discharge_disposition_id_5     uint8
discharge_disposition_id_6     uint8
discharge_disposition_id_7     uint8
discharge_disposition_id_8     uint8
discharge_disposition_id_9     uint8
d

In [12]:
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if 'readmitted_tf' in df_mlt_hot:
    y = df_mlt_hot['readmitted_tf'].values # get the labels we want
    del df_mlt_hot['readmitted_tf'] # get rid of the class label
    X = df_mlt_hot.values # use everything else to predict!

    ## X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    #    have converted them into simple matrices to use with scikit learn
    
    
# to use the cross validation object in scikit learn, we need to grab an instance
#    of the object and set it up. This object will be able to split our data into 
#    training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)

ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


In [20]:
#Model 5 - Log of 3 numericals: num_lab_procedures, num_medications, number_diagnoses

# create a reusable logisitic regression object
# here we can setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object

iter_num=0
# okay, so run through the cross validation loop and set the training and testing variable for one single iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]

# train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("Model 5 - Accuracy:", acc )
    print("confusion matrix\n",conf)
    iter_num+=1

====Iteration 0  ====
Model 5 - Accuracy: 0.6783924535717795
confusion matrix
 [[9018 2083]
 [4463 4790]]
====Iteration 1  ====
Model 5 - Accuracy: 0.6776554976908715
confusion matrix
 [[9004 1973]
 [4588 4789]]
====Iteration 2  ====
Model 5 - Accuracy: 0.6807507123906849
confusion matrix
 [[9019 1907]
 [4591 4837]]


In [24]:
# here we can change some of the parameters interactively
from ipywidgets import widgets as wd

def lr_explor(cost):
    lr_clf = LogisticRegression(penalty='l2', C=cost, class_weight=None) # get object
    accuracies = cross_val_score(lr_clf,X,y=y,cv=cv_object) # this also can help with parallelism
    print(accuracies)

wd.interact(lr_explor,cost=(0.001,5.0,0.05),__manual=True)

[0.6742655  0.67514985 0.67740985]


<function __main__.lr_explor(cost)>

In [25]:
# interpret the weights

# iterate over the coefficients
weights = lr_clf.coef_.T # take transpose to make a column vector
variable_names = df_mlt_hot.columns
for coef, name in zip(weights,variable_names):
    print(name, 'has weight of', coef[0])

age has weight of 0.043593889774561266
time_in_hospital has weight of -0.0011796584391227407
race_AfricanAmerican has weight of 0.1063090936154743
race_Asian has weight of -0.29371459996048815
race_Caucasian has weight of 0.15892216035990264
race_Hispanic has weight of -0.018018827748997976
race_Other has weight of -0.051245997025594625
race_Unspecified has weight of -0.2206716379048669
gender_Female has weight of -0.11810451453265529
gender_Male has weight of -0.20031529399626272
admission_type_id_1 has weight of -0.625837279304066
admission_type_id_2 has weight of -0.3603988004825984
admission_type_id_3 has weight of -0.7230604075428815
admission_type_id_4 has weight of -0.7737730664271127
admission_type_id_5 has weight of -0.7065613804294949
admission_type_id_6 has weight of -0.02378119766544122
admission_type_id_7 has weight of 3.4934881510323987
admission_type_id_8 has weight of -0.5984958277560859
discharge_disposition_id_1 has weight of 0.30509612383582296
discharge_disposition_

In [28]:
#Model 6 - Normalize Features with Log of 3 numericals: num_lab_procedures, num_medications, number_diagnoses
#Scale attributes by training set
scl_obj = StandardScaler()
scl_obj.fit(X_train)

x_train_scaled = scl_obj.transform(X_train) # apply to training
x_test_scaled = scl_obj.transform(X_test)

#Train the model just as before
lr_clf = LogisticRegression(penalty='l2', C=0.05) 
lr_clf.fit(x_train_scaled,y_train)

#Get test set predictions
y_pred6 = lr_clf.predict(x_test_scaled)

acc = mt.accuracy_score(y_test,y_pred6)
conf = mt.confusion_matrix(y_test,y_pred6)
print('Model 6 - Accuracy:', acc, '\nConfusion Matrix\n', conf)

#Sort these attributes and spit them out
zip_vars = zip(lr_clf.coef_.T,df_mlt_hot_x.columns) # combine attributes
zip_vars = sorted(zip_vars)
for coef, name in zip_vars:
    print(name, 'has weight of', coef[0]) # now print them out

Model 6 - Accuracy: 0.6807998427827454 
Confusion Matrix
 [[9018 1908]
 [4589 4839]]
discharge_disposition_id_11 has weight of -0.8555396856929514
discharge_disposition_id_14 has weight of -0.18853549296435587
discharge_disposition_id_13 has weight of -0.14238148553844976
admission_source_id_2 has weight of -0.11280748341485114
admission_type_id_4 has weight of -0.10218804094761405
admission_type_id_3 has weight of -0.09817899499020535
admission_source_id_1 has weight of -0.08228336063919839
medical_specialty_InternalMedicine has weight of -0.08093408041446774
medical_specialty_Surgery-Cardiovascular/Thoracic has weight of -0.07490395438209739
admission_source_id_4 has weight of -0.0689149696041042
admission_source_id_6 has weight of -0.06671797171738549
admission_source_id_17 has weight of -0.06541709437834951
admission_type_id_1 has weight of -0.057204107659377265
discharge_disposition_id_19 has weight of -0.0541221235756477
num_procedures has weight of -0.05183393118387993
medical_s